# Cross-Site Evaluation (with SAG)

In this example, we will demonstrate the Cross-Site Evaluation workflow using the Client API and the CIFAR10 dataset. In order to first produce models to perform cross-evaluation with, we run the [SAG workflow](../sag/sag.ipynb) beforehand.

## Cross-Site Evaluation Workflow

<img src="figs/cse.png" alt="cse" width=35% height=35% />

(Note: the diagram above illustrates evaluation on client-1's model, however in the workflow all participating clients will have their models evaluated by all other participating clients)

The `CrossSiteModelEval` workflow uses the data from clients to run evaluation with the models of other clients. Data is not shared, rather the collection of models is distributed by the server to each client site to run local validation. The server’s global model is also distributed to each client for evaluation on the client’s local dataset for global model evaluation. Finally, validation results are collected by the server to construct an all-to-all matrix of model performance vs. client dataset, and the `ValidationJsonGenerator` is used to write the results to a JSON file on the server.

Required tasks: 
- `validate` to perform validation on model using local dataset
- `submit_model` to obtain the client model for validation

## Converting DL training code to FL training code with Multi-Task Support
<a id = "code"></a>

We use the [Client API FL code](../code/fl/train.py) trainer, which is converted from the original [Training a Classifier](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) example.

Key changes when writing a FL code to support multiple tasks:
- When using the default `launch_once` parameter of `SubprocessLauncher`, we encapsulate our code in `while flare.is_running():` loop so we can call `flare.receive()` and perform various tasks. This is useful when launching everytime would be inefficient, such as when having to perform data setup every time.
- We use `flare.is_train()`, `flare.is_evaluate()`, and `flare.is_submit_model()` for implementing the `train`, `validate`, and `submit_model` tasks depending on the mode.

```
    # (3) run continously when launch_once=true
    while flare.is_running():

        # (4) receive FLModel from NVFlare
        input_model = flare.receive()

        # (5) performing train task on received model
        if flare.is_train():
            ...
        # (6) performing evaluate task on received model
        elif flare.is_evaluate():
            ...
        # (7) performing submit_model task to obtain best local model
        elif flare.is_submit_model():
            ...
```

See [Converting to FL code using Client API](../sag/sag.ipynb#code) for more details on using the Client API.

## Prepare Data

Make sure the CIFAR10 dataset is downloaded with the following script:

In [ ]:
! python ../data/download.py

## Job Configuration

Now we must configure our client api trainer along with the Server-Controlled Cross-site Evaluation workflows.

The client configuration for the trainer with the Client API is standard with the PTClientAPILauncherExecutor, SubprocessLauncher, and our defined app script that supports the `train`, `validate`, and `submit_model` tasks. 

In the server configuration, after `ScatterAndGather` we add the `CrossSiteModelEval` workflow, which uses the `validate` and `submit_model` tasks and requires a model locator. Under components the `PTFileModelLocator` is used to locate the models inventory saved during training, and an optional `IntimeModelSelector` is used to select the best global model to save based on the validation scores from the clients. Finally, the `ValidationJsonGenerator` generates `cross_val_results.json` which contains the accuracy of each validate model.

Let's copy the required files:

In [ ]:
! cp ../code/fl/train.py train.py
! cp ../code/fl/net.py net.py

Let's use the Job API to create a job and run using the simulator:

In [ ]:
from net import Net
from nvflare import FedJob
from nvflare.app_common.workflows.fedavg import FedAvg
from nvflare.app_common.workflows.cross_site_eval import CrossSiteEval
from nvflare.app_opt.pt.job_config.model import PTModel
from nvflare.job_config.script_runner import ScriptRunner


if __name__ == "__main__":
    n_clients = 2
    num_rounds = 1
    train_script = "train.py"

    job = FedJob(name="cse")
    
    # Define the initial global model and send to server
    comp_ids = job.to(PTModel(Net()), "server")
    
    # Define the controller workflow and send to server
    controller = FedAvg(
        num_clients=n_clients,
        num_rounds=num_rounds,
        persistor_id=comp_ids["persistor_id"]
    )
    job.to(controller, "server")

    # Define the controller workflow and send to server
    controller = CrossSiteEval(
        persistor_id=comp_ids["persistor_id"]
    )
    job.to(controller, "server")

    # Add clients
    for i in range(n_clients):
        runner = ScriptRunner(
            script=train_script,
            script_args="--local_epochs 1 --batch_size 32",
            launch_external_process=True            
        )
        job.to(runner, f"site-{i+1}")

    job.export_job("/tmp/nvflare/jobs")
    job.simulator_run("/tmp/nvflare/jobs/workdir", gpu="0")


## Run Job

The previous cell exports the job config and executes the job in NVFlare simulator.

If you want to run in production system, you will need to submit this exported job folder to nvflare system.


To view the validation results:

In [ ]:
! cat /tmp/nvflare/jobs/workdir/server/simulate_job/cross_site_val/cross_val_results.json

For additional resources, see other examples for SAG with CSE using the [Executor](../sag_executor/sag_executor.ipynb).

[Hello-Numpy](https://github.com/NVIDIA/NVFlare/tree/main/examples/hello-world/hello-numpy-cross-val) also demonstrates how to run cross-site evaluation using the previous training results.

Next we will look at the [cyclic](../cyclic/cyclic.ipynb) example, which shows the cyclic workflow for the Cyclic Weight Transfer algorithm.